In [78]:
import numpy as np 
import pandas as pd
import portion as P 
import string
import itertools
from fractions import Fraction
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable


# Finding reductions for linear problems
Consider the following locally verifiable problem in (d, $\delta$)-biregular trees:
- $\Sigma \subseteq [0,1]$
- Task is to label the edges
- Sum of edge labels incident to
    - white nodes is $\geq \alpha$
    - black nodes is $\leq \beta$
- Additionally leaf nodes accept all possible neighbourhoods.

This program tries to find a value $\{a, b, ...\}$ in each maximal separate continuous interval $A, B, C, ... \subseteq \Sigma = [0, 1]$, so that any valid labeling can be transformed to another valid labeling by **simultaneously** replacing all labels with the value corresponding to their interval. This reduces the set of labels down to $\Sigma = \{a, b, ...\}$.





The basic idea is to check every combination of these intervals and note which of these combinations could be neighbourhoods of white and/or black nodes. Using these combinations we can form a system of linear inequalities that the simultanious reductions must satisfy. Lastly we use linear programming to search for a set of values.


## Example 1:

Locally verifiable problem $\Pi$ in (3,3)-biregular trees:
- $\Sigma = [0, 1/3) \cup (2/3, 1]$
- $\alpha = 1$
- $\beta = 2$

Clearly there are no 0-round solutions, as $[1/3, 2/3] \cap \Sigma = \emptyset$. Let $A= [0, 1/3)$ and $B= (2/3, 1]$. The possible combinations for a neighbourhood are:

- A, A, A 
- A, A, B 
- A, B, B 
- B, B, B

We can also calculate the range of the sum of any given neighbourhood:

- $A, A, A = \left[0, 1 \right)$
- $A, A, B = \left(\frac{2}{3}, \frac{5}{3}\right)$
- $A, B, B = \left(\frac{4}{3}, \frac{7}{3}\right)$
- $B, B, B = \left(2, 3\right]$

This table shows us, for example, that that the neighbourhood $A, A, B$ can be a neighbourhood of a black or a white node. This however doesn't yet mean that **any** three values from the corresponding ranges can surround both white and black nodes. It just means that if we were to collapse these ranges into single values, that collapsed neighbourhood would need to be suitable for both white and black nodes. Using this idea we can create a system of inequalities that any suitable reduction must satisfy.

\begin{align*}

a \in A \iff 0\leq a &< \frac{1}{3} \\
b \in B \iff \frac{2}{3} < b &\leq 1 \\
3a &\leq 2 \\
1 \leq 2a+b  &\leq 2 \\
1 \leq a+2b &\leq 2 \\
3b &\geq 1
\end{align*}

Now any pair $(a,b)$ satisfying these inequalities will be a valid reducion. Using linear programming, we can find for example the solution $(0.161667, 0.676667)$. Now the problem just as easy as the LCL problem (in RE syntax):
```
A A A
A A B
A B B


A A B
A B B
B B B
```

Note that these reductions **cannot** be found separately: if we choose any value for $a\in A$, we can find values $b_1, b_2, b_3 \in B$ so that a white node has the neighbourhood $a+a+b_1<1$ or a black node has the neighbourhood $a+b_2+b_3>2$, which breaks our labeling. Same logic applies to fixing $b$ first.

In [79]:
# Define some variables

# Degrees of black and white nodes
d = 4
delta = 4

# The treshold for black and white sums
beta = Fraction(1, 1)
alpha = Fraction(9, 10)

# The set of possible labels
Sigma_string = "[0, 1/1000) U (999/1000, 1]"

# Value used to handle strict inequalities in calculations. Can affect possible results. (We approximate a<b  <=> a<=b-epsilon <=> b>= a+epsilon.)
epsilon = 0.000001

# Enable/disable debug prints
debug = False

In [80]:
# Define parser for Sigma
def convert(s):
    try:
        return float(s)
    except ValueError:
        return (s)

params = {
    'disj': ' U '
}

Sigma = P.from_string(Sigma_string, conv=Fraction, **params)

In [81]:
interval_count = len([x for x in Sigma])
intervals = pd.DataFrame({"interval": [x for x in Sigma], "reduction": [None for x in Sigma]}, index=list(string.ascii_uppercase[0:interval_count]))
if debug: print(intervals)

In [82]:
def sum_intervals(interval_list):
    min = 0
    max = 0
    min_in = True
    max_in = True
    for interval in interval_list:
        min_in = min_in and interval.left == P.CLOSED
        max_in = max_in and interval.right == P.CLOSED
        min += interval.lower
        max += interval.upper
    if min_in and max_in:
        return P.closed(min, max)
    if min_in and not max_in:
        return P.closedopen(min, max)
    if not min_in and max_in:
        return P.openclosed(min, max)
    return P.open(min, max)


In [83]:
# If white and black nodes have same amount of neighbours, some neighborhoods can be suitable for both
if d==delta:
    combinations = list(itertools.combinations_with_replacement(intervals.index, d))
    neighbourhoods = pd.DataFrame({"combination": combinations})
    neighbourhoods["W"] = None 
    neighbourhoods["B"] = None

else:
    combinations = list(itertools.combinations_with_replacement(intervals.index, delta))
    combinations.extend(list(itertools.combinations_with_replacement(intervals.index, d)))
    neighbourhoods = pd.DataFrame({"combination": combinations})
    neighbourhoods["OK"] = None
    
if debug: print(neighbourhoods)


In [84]:
white_range = P.closedopen(alpha, P.inf)
black_range = P.openclosed(-P.inf, beta)

if debug: 
    print(f"Black range: {black_range}")
    print(f"White range: {white_range}")
    print("\nRanges of neighbourhood sums")

for index, row in neighbourhoods.iterrows():
    range = sum_intervals(list(map(lambda x: intervals.loc[x, "interval"], row["combination"])))
    
    if debug:
        c = " ".join(row["combination"])
        print(f"{c}: {range}" )
    # See if we have to check both white and black ranges or only one
    if d==delta:
        neighbourhoods.at[index, "W"] = not (white_range & range).empty
        neighbourhoods.at[index, "B"] = not (black_range & range).empty
        
    else: 
        if len(row["combination"]) == d:
            neighbourhoods.at[index, "OK"] = not (black_range & range).empty
            
        else: neighbourhoods.at[index, "OK"] = not (white_range & range).empty
        
if debug: print(neighbourhoods)


In [85]:
def print_retor(neighbourhoods):
    if d == delta:
        white_retor = ""
        black_retor = ""
        for index, row in neighbourhoods.iterrows():
            if row["W"]:
                white_retor += " ".join(row["combination"])
                white_retor += "\n"
            if row["B"]:
                black_retor += " ".join(row["combination"])
                black_retor += "\n"

    else:
        white_retor = ""
        black_retor = ""
        for index, row in neighbourhoods.iterrows():
            if len(row["combination"]) == d and row["OK"]:
                black_retor += " ".join(row["combination"])
                black_retor += "\n"
            elif row["OK"]:
                white_retor += " ".join(row["combination"])
                white_retor += "\n"
    
    
    print("\nRound eliminator syntax:")
    print("\n"+black_retor)
    print("\n"+white_retor)

In [86]:
def find_reductions(neighbourhoods):
    model = LpProblem(name="Reductions", sense=LpMaximize)

    variables = dict(zip(list(string.ascii_lowercase[0:interval_count]), 
                    [LpVariable(name = symbol, lowBound=0, upBound=1) for symbol in list(string.ascii_lowercase[0:interval_count])]))
    
    

    # Add the constraints of the original sets
    for index, row in intervals.iterrows():
        variable = variables[index.lower()]
        interval = row["interval"]
        if interval.left == P.CLOSED:
            model += (variable>=interval.lower, f"Init_{index}_lower")
        else:
            model += (variable>=interval.lower + epsilon, f"Init_{index}_lower")
            
        if interval.right == P.CLOSED:
            model += (variable<=interval.upper, f"Init_{index}_upper")
        else:
            model += (variable<=interval.upper - epsilon, f"Init_{index}_upper")


    # Add the constraints imposed by summation of the original sets
    if d == delta:
        for index, row in neighbourhoods.iterrows():
            if (row["W"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) >= alpha, f"Constraint_{index}_W")
            if (row["B"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) <= beta, f"Constraint_{index}_B")

    else:  
        for index, row in neighbourhoods.iterrows():
            if (len(row["combination"])==d) and row["OK"]:
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) <= beta, f"Constraint_{index}_B")
            elif (row["OK"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) >= alpha, f"Constraint_{index}_W")


    if debug: print("\n", model, "\n")

    # Search for solutions to the given system of inequalities
    if model.solve() == -1:
        print("No reductions found.")
    
    else:
        print("Reductions found: ")
        for var in model.variables()[1:]:
            intervals.loc[var.name.upper(), "reduction"] = var.value()
        
        print(intervals)
        print_retor(neighbourhoods)
        


In [76]:
def run_reductor():
    # First check for 0-round solution
    easy_solution_interval = P.closed(Fraction(alpha, delta), Fraction(beta, d))
    easy_solutions = (easy_solution_interval & Sigma)
    if not easy_solutions.empty:
        print("0-round solution found.")
        print(f"Choose any single value from {P.to_string(easy_solutions, **params)}.")

    # Try to find some reductions
    else:
        print("No 0-round solutions found.")
        find_reductions(neighbourhoods)
    
    


In [77]:
run_reductor()

No 0-round solutions found.
Reductions found: 
                            interval reduction
A   [Fraction(1, 5),Fraction(9, 40))  0.216633
B  (Fraction(1, 4),Fraction(13, 50)]    0.2501

Round eliminator syntax:

A A A A
A A A B
A A B B
A B B B


A A A B
A A B B
A B B B
B B B B

